In [10]:
import numpy as np
import math

In [11]:
def fun1():
    Burntime=np.int(0)
    T=np.int(500) 
    L=16
    M=L
    N=L
    LF=0.1 
    z=4 
    r=2.3  
    B=0.12 
    GF=1 - LF 
    dt=200 
    #SamNum=np.int(T/dt)
    #Sample=np.empty((SamNum,M,N)) 
    modelstate=np.empty((M,N))
    MU=0.5
    SIGMA=0.1
    for i in range(M):
        for j in range(N):
            R1=np.random.normal(0.5,0.1)
            while R1<0:
                R1=np.random.normal(0.5,0.1)
            modelstate[i,j]=R1
    modelevolution=np.empty((M+2,N+2))
    modelevolution[1:M+1,1:N+1]=modelstate
    modelevolution[0,1:N+1]=modelstate[M-1,:]
    modelevolution[M+1,1:N+1]=modelstate[0,:] 
    modelevolution[1:N+1,0]=modelstate[:,N-1] 
    modelevolution[1:N+1,N+1]=modelstate[:,0]
    modelevolution[0,0]=modelstate[M-1,N-1]
    modelevolution[0,N+1]=modelstate[M-1,0]
    modelevolution[M+1,0]=modelstate[0,N-1]
    modelevolution[M+1,N+1]=modelstate[0,0]
    PreModelstate=modelstate
    PostModelstate=modelstate
    PreModelevolution=modelevolution
    PostModelevolution=modelevolution
    for k in range(Burntime+T):
        PreModelstate=PostModelstate
        PreModelevolution=PostModelevolution
        stateNoise=np.random.randn(M,N)
        evolutionNoise=np.empty((M+2,N+2))
        evolutionNoise[1:M+1,1:N+1]=stateNoise
        evolutionNoise[0,1:N+1]=stateNoise[M-1,:]
        evolutionNoise[M+1,1:N+1]=stateNoise[0,:] 
        evolutionNoise[1:N+1,0]=stateNoise[:,N-1] 
        evolutionNoise[1:N+1,N+1]=stateNoise[:,0]
        evolutionNoise[0,0]=stateNoise[M-1,N-1]
        evolutionNoise[0,N+1]=stateNoise[M-1,0]
        evolutionNoise[M+1,0]=stateNoise[0,N-1]
        evolutionNoise[M+1,N+1]=stateNoise[0,0]
        for i in range(1,M+1):
            for j in range(1,N+1):
                North=PreModelevolution[i-1,j] 
                South=PreModelevolution[i+1,j]
                East=PreModelevolution[i,j+1]
                West=PreModelevolution[i,j-1]
                Neighbour=np.array([North,South,East,West])
                North1=evolutionNoise[i-1,j]
                South1=evolutionNoise[i+1,j]
                East1=evolutionNoise[i,j+1]
                West1=evolutionNoise[i,j-1]
                Neighbour1=np.array([North1,South1,East1,West1])
                GlobalChange=PreModelevolution[i,j]*math.exp(r*(1-PreModelevolution[i,j]))*(1 + B*evolutionNoise[i,j]) #这里python的exp为标准库math的exp
                NeighbourChange=np.sum(Neighbour*np.exp(r*(1-Neighbour))*(1 + B*Neighbour1)) #这里python的exp为numpy的exp，也可以用array.sum()
                PostModelevolution[i,j]=GF*GlobalChange+(LF/z)*NeighbourChange
        PostModelstate=PostModelevolution[1:M+1,1:N+1]
        PostModelevolution[0,1:N+1]=PostModelstate[M-1,:]
        PostModelevolution[M+1,1:N+1]=PostModelstate[0,:] 
        PostModelevolution[1:N+1,0]=PostModelstate[:,N-1] 
        PostModelevolution[1:N+1,N+1]=PostModelstate[:,0]
        PostModelevolution[0,0]=PostModelstate[M-1,N-1]
        PostModelevolution[0,N+1]=PostModelstate[M-1,0]
        PostModelevolution[M+1,0]=PostModelstate[0,N-1]
        PostModelevolution[M+1,N+1]=PostModelstate[0,0]